In [2]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings("default")
import sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.model_selection import cross_val_score, StratifiedKFold
import matplotlib.pyplot as plt
from sklearn import model_selection
import time
from sklearn.neural_network import MLPRegressor

pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000

C:\Anaconda\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Anaconda\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Anaconda\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Anaconda\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [72]:
df_merge = pd.read_excel('prepared_hard.xlsx')

In [73]:
ls_steel = df_merge['марка стали'].value_counts().index
len(ls_steel)

64

In [65]:
col_chem = [
     'C', 
    'Mn', 
    'Si', 
    'P', 
    'S',
    'Cr', 
    'Ni', 
    'Cu', 
    'Al', 
    'V', 
    'Ti', 
    'Nb', 
    'Mo', 
    'N', 
    'B'
]

ls = col_chem + ['марка стали']

In [66]:
def mean_chem(df, steel):
    df = df[df['марка стали'] == steel]
    df[col_chem] = df.groupby(['марка стали'])[col_chem].transform(lambda x: round(x.mean(), 5))
    df = df.reset_index(drop=True)
    df = df.drop_duplicates('C')
    df = df[ls]
    return df

In [67]:

df_merge_1 = df_merge[df_merge['марка стали'] == '37Г2Ф']
df_merge_1[col_chem] = df_merge_1.groupby('марка стали')[col_chem].transform(lambda x: round(x.mean(), 5))
df_merge_1 = df_merge_1.reset_index(drop=True)
df_merge_1 = df_merge_1.drop_duplicates('C')
df_merge_1 = df_merge_1[ls]
df_merge_1

C:\Anaconda\lib\site-packages\pandas\core\frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,B,марка стали
0,0.36306,1.49444,0.28281,0.01123,0.00803,0.10424,0.10694,0.14267,0.0304,0.0878,0.00321,0.00014,0.00072,0.00626,0.0,37Г2Ф


In [68]:
df_merge_2 = mean_chem(df_merge, '30Г2')
df_merge_2

C:\Anaconda\lib\site-packages\pandas\core\frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,B,марка стали
0,0.28349,1.33282,0.25721,0.00963,0.00717,0.1032,0.12547,0.1932,0.02672,0.0048,0.00711,0.00574,0.01597,0.00837,0.0,30Г2


In [69]:
df_tmp = pd.merge(df_merge_1, df_merge_2, how = 'outer')
df_tmp

,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,B,марка стали
0,0.36306,1.49444,0.28281,0.01123,0.00803,0.10424,0.10694,0.14267,0.03040,0.0878,0.00321,0.00014,0.00072,0.00626,0.0,37Г2Ф
1,0.28349,1.33282,0.25721,0.00963,0.00717,0.10320,0.12547,0.19320,0.02672,0.0048,0.00711,0.00574,0.01597,0.00837,0.0,30Г2


In [70]:
df_merge_steel = df_tmp
for steel in ls_steel: 
    df_merge_steel =  pd.merge(df_merge_steel, mean_chem(df_merge, steel), how = 'outer')

df_merge_steel

C:\Anaconda\lib\site-packages\pandas\core\frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,B,марка стали
0,0.36306,1.49444,0.28281,0.01123,0.00803,0.10424,0.10694,0.14267,0.03040,0.08780,0.00321,0.00014,0.00072,0.00626,0.00000,37Г2Ф
1,0.28349,1.33282,0.25721,0.00963,0.00717,0.10320,0.12547,0.19320,0.02672,0.00480,0.00711,0.00574,0.01597,0.00837,0.00000,30Г2
2,0.14530,0.53686,0.27402,0.00838,0.00412,0.56769,0.08677,0.11189,0.03490,0.05290,0.00178,0.00358,0.00961,0.00615,0.00013,13ХФА
3,0.17664,0.57513,0.27010,0.00840,0.00579,0.99040,0.12384,0.17346,0.03250,0.07295,0.01334,0.03993,0.13883,0.00829,0.00000,18ХМФБ
4,0.28363,1.32279,0.26187,0.00883,0.00661,0.09805,0.12095,0.18601,0.02641,0.00579,0.00748,0.00725,0.01034,0.00844,0.00000,30Г2-2
5,0.27739,1.35186,0.25759,0.00879,0.00617,0.10920,0.13565,0.19184,0.02898,0.00418,0.00975,0.00450,0.01913,0.00999,0.00000,30Г2-8
6,0.38342,1.57563,0.50397,0.01073,0.00772,0.11773,0.12552,0.18278,0.02762,0.03459,0.00012,0.00003,0.00110,0.00792,0.00001,38Г2С
7,0.09577,1.38216,0.58760,0.01142,0.01013,0.08400,0.09545,0.13071,0.02888,0.00651,0.00755,0.00281,0.01174,0.00765,0.00004,09Г2С
8,0.29016,1.35889,0.26241,0.00953,0.00654,0.09381,0.11791,0.20802,0.02710,0.00562,0.00899,0.00619,0.01231,0.00902,0.00000,30Г2-3
9,0.32904,0.62752,0.26412,0.01140,0.01019,0.91392,0.08614,0.12379,0.02891,0.00248,0.00091,0.00000,0.00323,0.00061,0.00000,32ХА


In [71]:
df_merge_steel.to_excel('mean_chem_steel.xlsx')